In [1]:
import pandas as pd
import pickle
import os
from os.path import join
import re
import tqdm
from file_function import re_name_upload, write_log_csv_upload
from tqdm import tqdm

##### 채무자조회.pkl 만들기

In [3]:
# 현재 버전 221118
path = r"C:\Users\SL\Desktop\workspace\python\fileNaming\파일"
excel_file = "채무자조회.xlsx"
df = pd.read_excel(join(path, excel_file))
len(df)

70792

In [4]:
pkl_file = "채무자조회.pkl"
df.to_pickle(join(path, pkl_file))

##### 신규 종결건 처리하기

In [5]:
df_old = pd.read_pickle(r"C:\Users\SL\Desktop\workspace\python\fileNaming\파일\채무자조회 구버전\채무자조회_221021.pkl").fillna("")
df_new = pd.read_pickle(r"C:\Users\SL\Desktop\workspace\python\fileNaming\파일\채무자조회.pkl").fillna("")

In [6]:
df1 = df_old.loc[:, ["채무자키", "채무상태"]]
df2 = df_new.loc[:, ["채무자키", "채무상태", "매각사구분"]]

In [9]:
df3 = pd.merge(df1,df2, on="채무자키", how="outer")
df3.columns = ["채무자키", "old", "new", "매각사구분"]
df3

,채무자키,old,new,매각사구분
0,20495151,정상,정상,전북-01
1,20495152,정상,정상,전북-01
2,20495238,정상,정상,전북-01
3,20417854,정상,정상,DNP-01
4,20411449,정상,정상,DNP-01
...,...,...,...,...
70787,20525075,NaN,신용회복(확정),카카오-01
70788,20526239,NaN,신용회복(확정),카카오-01
70789,20527282,NaN,신용회복(확정),카카오-01
70790,20525073,NaN,개인회생(확정),카카오-01


In [17]:
df4 = df3[~(df3["old"].str.match(r"개인회생\(면책\)|파산\(면책\)|환매|매각|종결", na=False)) & (df3["new"].str.match(r"개인회생\(면책\)|파산\(면책\)|환매|매각|종결", na=False))]
df4

,채무자키,old,new,매각사구분
67,20495134,정상,종결,전북-01
480,20432765,정상,파산(면책),KB카드-01
579,20433694,정상,파산(면책),KB카드-01
941,20502810,정상,종결,스마트저축-01
1397,20421481,정상,종결,한울가람
...,...,...,...,...
67263,20441028,정상,파산(면책),KB카드-01
67499,20431563,정상,종결,KB카드-01
67668,20408935,개인회생(확정),개인회생(면책),대산
70116,20527419,NaN,종결,카카오-01


In [21]:
#보고용
# df5 = df3[df3["new"].str.match(r"개인회생\(면책\)|파산\(면책\)|환매|매각|종결")]["채무자키"]
# df5.index = range(len(df5))
# df5.to_excel("c://Users/SL/Desktop/관리제외221021.xlsx")

In [22]:
# 신규 종결건 보고용
# df4.columns = ["채무자키", "채무상태220906", "채무상태221021", "매각사구분"]
# df4.reset_index(inplace=True, drop=True)
# df4.to_excel("c://Users/SL/Desktop/신규종결건.xlsx")

In [18]:
result = [(str(x), y) for x, y in zip(df4.채무자키, df4.매각사구분)]
len(result)

296

In [19]:
docu_folder_dict = {"원인서류": "1.원인서류", "양도통지서": "2.양도통지서", "집행권원": "3.집행권원", "강제집행": "4.강제집행", 
        "등본": "5.등초본", "법인등기" : "5.등초본", "초본": "5.등초본", "등초본":"5.등초본", "외국인증명": "5.등초본", "주민등록정보":"5.등초본", 
        "개인회생": "6.개인회생", "신용회복": "7.신용회복", "파산": "8.파산", "재산조사": "9.재산조사", "부채증명서" : "10.부채증명서", 
        "신용조회" : "11.신용조회", "개인정보":"12.개인정보", "기타": "기타"}
depth1 = list(docu_folder_dict.values())
depth1

['1.원인서류',
 '2.양도통지서',
 '3.집행권원',
 '4.강제집행',
 '5.등초본',
 '5.등초본',
 '5.등초본',
 '5.등초본',
 '5.등초본',
 '5.등초본',
 '6.개인회생',
 '7.신용회복',
 '8.파산',
 '9.재산조사',
 '10.부채증명서',
 '11.신용조회',
 '12.개인정보',
 '기타']

###### 신규 종결건 파일서버에서 삭제하기

In [20]:
# path에 신규 종결건 파일이 있는 경우만 골라서 관리제외 폴더로 옮기기
path = r"\\192.168.0.75\솔림헬프"
out_dir_base = r"\\192.168.0.75\삭제예정파일\관리제외"
path_log = r'\\192.168.0.75\스캔파일\스캔파일log\_project\파일\채무자조회 구버전'
success = []
fail = []
dir_cnt = 0

for d1 in tqdm(depth1) : 
    for d3, d2 in result :
        src_d = join(path, d1, d2, d3)
        if os.path.exists(src_d) :

            dst_d = join(out_dir_base, d2, d3)
            if not os.path.exists(dst_d) :
                os.makedirs(dst_d)

            for f in os.listdir(src_d) :  # 파일 옮기기
                try :
                    success.append(re_name_upload(join(src_d, f), join(dst_d, f)))
                except Exception as e :    
                        fail.append([src_d, f, dst_d, f, e.__class__])
                        continue
            
            try : 
                os.rmdir(src_d) # 빈폴더 삭제하기
                dir_cnt += 1
            except Exception as e : 
                print(e)
            
print(len(success),"개의 파일 이동")
print(len(fail),"개의 파일 이동 실패")
print(dir_cnt,"개의 폴더 삭제")
write_log_csv_upload(success, path_log, "success")
write_log_csv_upload(fail, path_log, "fail")

100%|██████████| 18/18 [21:30<00:00, 71.68s/it]  


1308 개의 파일 이동
0 개의 파일 이동 실패
799 개의 폴더 삭제
log 파일이 비어있습니다. path : \\192.168.0.75\스캔파일\스캔파일log\_project\파일\채무자조회 구버전


In [23]:
dir_cnt

799

In [109]:
# 전체 os.walk 하는 비효율적인 코드
# path = r"\\192.168.0.75\솔림헬프"
# out_dir_base = r"\\192.168.0.75\관리제외"
# path_log = r'\\192.168.0.75\스캔파일\스캔파일log\_project\파일\채무자조회 구버전'
# success = []
# fail = []
# dir_cnt = 0
# for root, __dirs__, files in tqdm(os.walk(path)) :
#     if re.search("#recycle", root) == None : # 휴지통 nono
#         for key in result :
#             try : 
#                 if re.search(key, root) :
#                     depth2 = root.split(os.path.sep)[-2]
#                     depth3 = root.split(os.path.sep)[-3]
#                     out_dir = join(out_dir_base, depth2, depth3)
                    
#                     # if not os.path.exists(out_dir) :
#                     #     os.makedirs(out_dir)
#                     try :
#                         for f in os.listdir(root) :
#                             success.append([join(root, f), join(out_dir, f)])
#                             # success.append(re_name_upload(join(root, f), join(out_dir, f)))
#                     except Exception as e :    
#                         fail.append([root, f, out_dir, f, e.__class__])
#                         continue
            
#                     # os.rmdir(out_dir)
#                     # dir_cnt += 1
#                     break
#             except Exception as e : 
#                 print(e.with_traceback())
#                 continue
                
# print(len(success),"개의 파일 이동")
# print(len(fail),"개의 파일 이동 실패")
# print(dir_cnt,"개의 폴더 삭제")
# write_log_csv_upload(success, path_log, "success")
# write_log_csv_upload(fail, path_log, "fail")

453 개의 파일 이동
0 개의 파일 이동 실패
0 개의 폴더 삭제
log 파일이 비어있습니다. path : \\192.168.0.75\스캔파일\스캔파일log\_project\파일\채무자조회 구버전
